In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
max_probability_of_contact = 0.2
max_probability_of_visiting_after_infected = 0.005
max_probability_of_visiting_unsuspicious_shop = 0.03

In [ ]:
def logistic_probability_same_time(number_of_minutes):
    K=max_probability_of_contact
    P=0.01
    r=0.1
    return (K*P)/(P+(K-P)*np.exp(-r*number_of_minutes))

def logistic_probability_same_place_from_initial(number_of_minutes):
    K=0.001
    P=max_probability_of_visiting_after_infected
    r=0.0001
    return (K*P)/(P+(K-P)*np.exp(-r*number_of_minutes))

def logistic_probability_visiting_a_shop(number_of_minutes):
    K=max_probability_of_visiting_unsuspicious_shop
    P=0.001
    r=0.1
    return (K*P)/(P+(K-P)*np.exp(-r*number_of_minutes))

def compute_final_score(shop_visits_with_infected=[10, 30], random_shop_visits=[10, 60], visits_after_infected=[380, 120]):
    not_infected_probs_actions = []
    for passed_time in shop_visits_with_infected:
        not_infected_probs_actions.append(1-logistic_probability_same_time(passed_time))
    for passed_time in random_shop_visits:
        not_infected_probs_actions.append(1-logistic_probability_visiting_a_shop(passed_time))
    for passed_time in visits_after_infected:
        not_infected_probs_actions.append(1-logistic_probability_same_place_from_initial(passed_time))
    print(not_infected_probs_actions)
    return 1 - np.prod(not_infected_probs_actions)

In [ ]:
x = np.linspace(0, 120)
y = logistic_probability_same_time(x)

In [ ]:
plt.plot(x, y)
plt.title('Probability of infection while in the same building over time')
plt.show()

In [ ]:
x = np.linspace(0, 24*60)
y = logistic_probability_same_place_from_initial(x)

In [ ]:
plt.plot(x, y)
plt.title('Probability of infection visiting place after infected person over time')
plt.show()

In [ ]:
x = np.linspace(0, 120)
y = logistic_probability_visiting_a_shop(x)

In [ ]:
plt.plot(x, y)
plt.title('Probability of infection visiting place over time')
plt.show()

In [ ]:
covid_cases_data_path = 'covid-cases-coordinates.csv'

In [ ]:
import pandas as pd
from datetime import datetime
user_data = pd.read_csv('user_data.csv')
covid_data = pd.read_csv(covid_cases_data_path, encoding='latin')

In [ ]:
def get_visited_public_places(user_data, past_number_of_weeks):
    curr_date = datetime.now()
    user_data['number_of_days_passed'] = user_data['visit_start_time'].apply(lambda x: (curr_date - datetime.fromtimestamp(x/1000)).days)
    user_data = add_required_time_information(user_data)
    home_addresses = get_possible_work_and_home_places(user_data)
    visited_public_places = get_public_places_visited_per_weeks(user_data, past_number_of_weeks, home_addresses)
    return visited_public_places

In [ ]:
def get_public_places_visited_per_weeks(user_data, past_number_of_weeks, home_addresses):
     user_data = get_last_days_data(user_data, past_number_of_weeks*7)
     return user_data[~user_data['name'].isin(home_addresses)]

In [ ]:
def get_last_days_data(user_data, number_of_days):
    return user_data[user_data['number_of_days_passed'] <= number_of_days].sort_values('number_of_days_passed')

In [ ]:
def get_possible_work_and_home_places(user_data):
    long_time_locations = user_data[user_data['time_spent_in_location'] > 4]['name'].unique()
    percentage_of_time_spent_per_location = user_data.groupby('name').apply(lambda x: sum(x['time_spent_in_location'])).sort_values()/sum(user_data['time_spent_in_location'])
    work_home_addresses = [location for location in long_time_locations if percentage_of_time_spent_per_location[location] > 0.1]
    print(work_home_addresses)
    return work_home_addresses

In [ ]:
def add_required_time_information(user_data):
    user_data['time_spent_in_location'] = user_data.apply(lambda x: (datetime.fromtimestamp(x['visit_end_time']//1000) - datetime.fromtimestamp(x['visit_start_time']//1000)).seconds/60, axis=1)
    return user_data

In [ ]:
visited_places = get_visited_public_places(user_data, 4)

In [ ]:
max_probability_of_contact = 0.2
max_probability_of_visiting_after_infected = 0.005
max_probability_of_visiting_unsuspicious_shop = 0.003

In [ ]:
compute_final_score(shop_visits_with_infected=[], random_shop_visits=visited_places['time_spent_in_location'].tolist(), visits_after_infected=[200])